In [1]:
! pip install seaborn

In [1]:
 # Libs principais
from model import *
import pandas as pd
# Controle de execução e pastas
import os
# Desativar alguns warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.style.use('ggplot')

def VirtualLaneDetector(x,y,virtual_lane_lim,virtual_lane_width):
    virtual_lane = -1
    count = 0

    for list_point in virtual_lane_lim:
        count = count + 1
        func_virtual_lane = PolygonalToFunction(list_point)
        if (y>=func_virtual_lane(x)-0.5*virtual_lane_width) and (y<=func_virtual_lane(x)+0.5*virtual_lane_width):
            virtual_lane = count
            break

    return virtual_lane

In [2]:
os.chdir("data_ignore")

In [5]:
vva = [3,4]
traffic_lanes_valid = [1,2,3,4,5]
id = "Piloto2_Drone2"
# os.chdir("data_ignore")
root_file = "data/json"
all_files = os.listdir(root_file)
all_files = [i for i in all_files if id in i]

for f in all_files:
    # try:
    print(f"Processando {f}")
    model = YoloMicroscopicDataProcessing()
    model.ImportFromJSON(f"data/json/{f}")

    # Mantém só uma das pistas
    model.df = model.df[model.df[model.traffic_lane_column].isin(traffic_lanes_valid)]

    # Remover dados das extreminades do vídeo
    model.df = model.df[model.df['x'].between(3,model.video_width-3)]

    # Alterar a frequência da amostra para 10fps
    min_frame = model.df[model.frame_column].min()
    max_frame = model.df[model.frame_column].max()
    list_frames = list(range(min_frame,max_frame+1,3))
    model.df = model.df[model.df[model.frame_column].isin(list_frames)]

    # Dataframe de motocicletas
    df_motorcycle = model.df[model.df[model.vehicle_type_column].isin(['Moto'])].sort_values([model.frame_column,model.id_column])
    df_motorcycle.insert(0,'id_voo',f.split(".")[0])

    # Definição das faixas de tráfego
    # Centroide (y) dentro da região cujo centroo é o limite entre as faixas
    # Largura teórica do corredor (metros), corresponde a largura de 2 motos
    virtual_lane_width = 1.6
    # Corredor sem faixa azul
    virutal_lane_group = {'Corredor Principal':vva}
    virutal_lane_group["Outros Corredores"] = [i for i in  range(1,len(model.virtual_lane_lim)+1) if i not in virutal_lane_group['Corredor Principal']]

    # Qual corredor virual pertence (-1 para nenhum corredor)
    df_motorcycle['virutal_lane'] = df_motorcycle.apply(lambda row:VirtualLaneDetector(row[model.x_centroid_column],row[model.y_centroid_column],model.virtual_lane_lim,virtual_lane_width),axis=1)
    # Corredores não classificados recebem 0
    df_motorcycle['zero_temp'] = ((-df_motorcycle['virutal_lane'].isin(JoinList(list(virutal_lane_group.values())))) & (df_motorcycle['virutal_lane']!=-1))
    df_motorcycle['virutal_lane'] = df_motorcycle.apply(lambda x:x['virutal_lane'] if not x['zero_temp'] else 0,axis=1)
    df_motorcycle = df_motorcycle.drop(columns=['zero_temp'])

    # Tipo/nome do corredor
    df_motorcycle['virtual_lane_type'] = np.nan
    for key,value in virutal_lane_group.items():
        df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:key if x['virutal_lane'] in value else x['virtual_lane_type'],axis=1)
    # Se for -1, estava na mais centralizado na faixa de tráfefo misto
    df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:'Fora do Corredor' if x['virutal_lane']==-1 else x['virtual_lane_type'],axis=1)
    # Se estava em outras faixas não avaliadas
    df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:'Outro Corredor' if x['virutal_lane']==0 else x['virtual_lane_type'],axis=1)

    # Condição do tráfego
    speed_ref = '85%'
    agg_traffic_state = model.df[-model.df[model.vehicle_type_column].isin(['Moto'])].groupby(['frame'])['instant_speed'].describe(percentiles=[0.85])
    agg_traffic_state = agg_traffic_state.reset_index(drop=False)
    df_motorcycle = df_motorcycle.merge(agg_traffic_state[['frame',speed_ref]],on='frame',how='left')
    df_motorcycle['traffic_condition_speed'] = df_motorcycle[speed_ref]*3.6
    df_motorcycle = df_motorcycle.drop(columns=[speed_ref])
    df_motorcycle['traffic_condition'] = df_motorcycle['traffic_condition_speed'].apply(lambda x:'Congestionado' if x<5 else 'Não Congestionado')

    # Distância lateral
    side_distance = pd.DataFrame()
    for motorcycle_id in df_motorcycle[model.id_column].unique().tolist():
        frame_list = df_motorcycle[df_motorcycle[model.id_column]==motorcycle_id].sort_values(model.frame_column)[model.frame_column].tolist()
        df_side = pd.concat([model.SideVehicle(
            motorcycle_id,
            t,
            overlap_lon=0.3,
            overlap_lat=0.3,
            report_just_min=True,
            ) for t in frame_list],ignore_index=True)
        df_side = df_side.rename(columns=dict(zip(df_side.columns, [i+'_vehicle_side' if 'speed' in i else i for i in df_side.columns])))
        df_side.insert(0,'id_motorcycle',motorcycle_id)
        side_distance = pd.concat([side_distance,df_side],ignore_index=True)

    side_distance = side_distance.rename(columns={'id':'id_vehicle_side'})
    side_distance[model.global_id_column] = side_distance['id_motorcycle'].astype(str) + '@' + side_distance[model.frame_column].astype(str)
    df_motorcycle = df_motorcycle.merge(side_distance[[
        'global_id',
        'id_vehicle_side',
        'x_instant_speed_vehicle_side',
        'y_instant_speed_vehicle_side',
        'instant_speed_vehicle_side',
        'lateral_distance_between_vehicles',
        'side'
        ]],on=model.global_id_column,how='left')

    df_motorcycle['delta_speed_vehicle_side'] = df_motorcycle[model.instant_speed_column] - df_motorcycle[model.instant_speed_column+'_vehicle_side']
    df_motorcycle['delta_speed_vehicle_side'] = df_motorcycle['delta_speed_vehicle_side']

    df_headway = []
    for index,row in df_motorcycle.iterrows():

        try:
            hd = model.HeadwayDeltaSpeed(row["id"],row["frame"])
            

            if not hd.empty:
                df_headway.append(hd)
        except Exception as e:
            print("Deu ruim pulou",row["id"],row["frame"])
    print("OK")
    
    if len(df_headway)>0:
        df_headway = pd.concat(df_headway,ignore_index=True)
        df_headway["id"] = df_headway["id_follower"]
        df_motorcycle = df_motorcycle.merge(df_headway,on=["frame","id"],how="left")

    df_motorcycle.to_csv(f"data/DistLatVelAceHeadway/{f.split('.')[0]}.csv",index=False)
    print(f"Fim {f}")
    # except Exception as e:
    #     print(f"Erro {f}")
    #     print(e)
        

Processando Piloto2_Drone2_0002.json
Deu ruim pulou 643 16026
OK
Fim Piloto2_Drone2_0002.json
Processando Piloto2_Drone2_0003.json
OK
Fim Piloto2_Drone2_0003.json
Processando Piloto2_Drone2_0004.json
OK
Fim Piloto2_Drone2_0004.json
Processando Piloto2_Drone2_0005.json
Deu ruim pulou 197 2973
OK
Fim Piloto2_Drone2_0005.json
Processando Piloto2_Drone2_0006.json
OK
Fim Piloto2_Drone2_0006.json
Processando Piloto2_Drone2_0007.json
OK
Fim Piloto2_Drone2_0007.json


#### Suavizar

In [5]:
if __name__=="__main__":
    # os.chdir("data_ignore")
    root_file = "data/json"
    all_files = os.listdir(root_file)

    for f in all_files:
        try:
            print(f"Processando {f}")
            model = YoloMicroscopicDataProcessing()
            model.ImportFromJSON(os.path.join(root_file,f))
        
            model_smoothed = model.SmoothingSavGolFilter(window_length=15,polyorder=1) 
            model_smoothed.to_csv(f"data/processed_smoothed/{f.replace('json','csv')}",index=False)
            print(f"Fim {f}")
        except Exception as e:
            print(f"Erro {f}")
            print(e)


Processando 10_A_1.json
Fim 10_A_1.json
Processando 10_A_2.json
Fim 10_A_2.json
Processando 10_A_3.json
Fim 10_A_3.json
Processando 10_A_4.json
Fim 10_A_4.json
Processando 10_A_5.json
Fim 10_A_5.json
Processando 10_B_1.json
Fim 10_B_1.json
Processando 10_B_2.json
Fim 10_B_2.json
Processando 10_B_3.json
Fim 10_B_3.json
Processando 10_B_4.json
Fim 10_B_4.json
Processando 10_B_5.json
Fim 10_B_5.json
Processando 32_A_1.json
Fim 32_A_1.json
Processando 32_A_2.json
Fim 32_A_2.json
Processando 32_A_3.json
Fim 32_A_3.json
Processando 32_A_4.json
Fim 32_A_4.json
Processando 32_A_5.json
Fim 32_A_5.json
Processando 32_B_1.json
Fim 32_B_1.json
Processando 32_B_2.json
Fim 32_B_2.json
Processando 32_B_3.json
Fim 32_B_3.json
Processando 32_B_4.json
Fim 32_B_4.json
Processando 32_B_5.json
Fim 32_B_5.json
Processando 79_A_1.json
Fim 79_A_1.json
Processando 79_A_2.json
Fim 79_A_2.json
Processando 79_A_3.json
Fim 79_A_3.json
Processando 79_A_4.json
Fim 79_A_4.json
Processando 79_A_5.json
Fim 79_A_5.json
